# Orbital Elements from Position and Velocity
##### Vallado Algorithm 9 (from pg 113)
##### RV2COE(r, v --> a, e, i, capOmega, omega, nu)

****
Inputs: 
* r: position vector (km)
* v: velocity vector (m/s)

****

Outputs:
- a: semimajor axis (km)   (SIZE)
- e: eccentricity          (SHAPE)
- i: inclination             (TILT)
- capOmega: longitude of ascending node  (PIN)
- omega: argument of periapsis     (TWIST)
- nu: true anomaly         (ANGLE NOW)

In [159]:
import numpy as np
import scipy.optimize as scio
import matplotlib.pyplot as plt
import math

In [186]:
## Global variables and constants

# Standard gravitational parameter of Earth (3.986*10^14 m^3*s^-2)
# Multiplied by 10e-9 to convert to km^3*s^-2
mu = 3.986004418 * 10.0**14 * (10**(-9)) # km^3 / s^2

K = [0, 0, 1]

In [187]:
def angular_momentum(position,velocity):
    
    r = position
    v = velocity
    
    h = np.cross(r,v)
    
    return h

In [188]:
def node_vector(angular_momentum):
    
    h = angular_momentum
    n_hat = np.cross(K,h)
    
    return n_hat

In [189]:
def eccentricity_v(position, velocity):

    r = position
    v = velocity
    
    a = 1.0 / mu
    b = np.multiply(((np.linalg.norm(v) ** 2)-(mu / np.linalg.norm(r))) , r)
    c = np.multiply((np.dot(r, v)) , v)
    d = b-c
    i = np.multiply(d, a)
    
    ev = i
    
    return ev

In [190]:
def specific_mechanical_energy(position, velocity):

    r = position
    v = velocity
    
    mech_e = (np.linalg.norm(v)**2)/2 - (mu/np.linalg.norm(r))
    
    return mech_e

In [191]:
def elements_from_r_v(position, velocity):

    r = position
    v = velocity

    h = angular_momentum(r, v)
    n = node_vector(h)

    ev = eccentricity_v(r, v)
    e = np.linalg.norm(ev)
    
    E = specific_mechanical_energy(r, v)
    
    
    if e != 1.0:
        a = -mu / (2 * E)
        p = a * (1 - e ** 2)
    else: 
        a = np.inf
        p = h ** 2 / mu
        
    #Inclination
    i = (180/math.pi)*np.arccos(h[2]/np.linalg.norm(h))
    
    #Longitude of Ascending Node
    omega = (180/math.pi)*np.arccos(n[0]/np.linalg.norm(n))
    
    if n[1] < 0:
        omega = 360 - omega
    
    #Argument of Periapsis
    argp = (180/math.pi)*np.arccos((np.dot(ev, n))/(np.linalg.norm(ev)*np.linalg.norm(n)))
    
    if ev[2] < 0:
        argp = 360 - argp
    
    # True Anomaly
    nu = (180/math.pi)*np.arccos((np.dot(ev, r))/(np.linalg.norm(ev)*np.linalg.norm(r))) 
    
    if (np.dot(r, v)) < 0:
        nu = 360 - nu
        
    return a, e, i, omega, argp, nu

In [216]:
def printElements(orbElements):
    
    # Convert tuple to list
    orbElements = list(orbElements)
    
    # Round each element to three decimals
    for i, element in enumerate(orbElements):
        orbElements[i] = float("{:.3f}".format(element))
    
    print('Semi major axis: ' + str(orbElements[0]) + ' km')
    print('Eccentricity: ' + str(orbElements[1]))
    print('Incline: ' + str(orbElements[2]) + ' degrees')
    print('Longitude of Ascending Node: ' + str(orbElements[3]) + ' degrees')
    print('Argument of Periapsis: ' + str(orbElements[4]) + ' degrees')
    print('True Anomaly: ' + str(orbElements[5]) + ' degrees')

In [217]:
# Test verification with example on page 114 of Vallado book
r = [6524.834, 6862.875, 6448.296]
v = [4.901327, 5.533756, -1.976341]

orbElements = elements_from_r_v(r, v)
printElements(orbElements)

Semi major axis: 36127.338 km
Eccentricity: 0.833
Incline: 87.869 degrees
Longitude of Ascending Node: 227.898 degrees
Argument of Periapsis: 53.385 degrees
True Anomaly: 92.335 degrees
